# Import data and functions

In [1]:
%tensorflow_version 2.x
import os
os.environ['PYTHONHASHSEED']=str(1)

import tensorflow as tf
import numpy as np
import random

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)

def reset_random_seeds(n=1):
   os.environ['PYTHONHASHSEED']=str(n)
   tf.random.set_seed(n)
   np.random.seed(n)
   random.seed(n)

# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from scipy.io import loadmat
from tensorflow import keras
from tensorflow.keras import layers
import gc
from sklearn.metrics import r2_score


def stagger_data(data, h):
    """
    >>> i = np.array([[1, 2, 3, 4, 5, 6], [7, 8, 9, 10, 11, 12]])
    >>> stagger_data(i, [1, 3])
    (array([[ 3,  4,  5],
           [ 9, 10, 11],
           [ 1,  2,  3],
           [ 7,  8,  9]]), array([[ 4,  5,  6],
           [10, 11, 12]]))
    """
    h.sort()
    len_h = len(h)
    n, m = data.shape
    max_h = max(h)

    Y = data[:, max_h:]
    X = np.zeros((n * len_h, m - max_h), dtype=data.dtype)
    for i in range(len_h):
        X[i * n: i * n + n, :] = data[:, max_h - h[i]:m - h[i]]
    return X, Y


def remove_weekends(data, start=0, bs=36):
    _, m = data.shape
    n_day = int(m / bs)
    weekday = np.concatenate([np.arange(start, 7) % 7, np.arange(n_day) % 7])[:n_day]
    weekday = np.repeat(weekday, bs)
    return data[:, weekday < 5]


def get_flow1(od, s, dir='o', num_s=159):
    """Get the flow of station `s`"""
    n = od.shape[0]
    if dir == 'o':
        idx = np.arange(s, n, num_s)
    elif dir == 'd':
        idx = np.arange((s * num_s), (s * num_s + num_s))
    return np.sum(od[idx, :], axis=0)


def od2flow(od, s_list=None, dir='o', num_s=159):
    if s_list is None:
        s_list = range(num_s)

    n_s = len(s_list)
    flow = np.zeros((n_s, od.shape[1]), dtype=np.float32)
    for i, s in enumerate(s_list):
        flow[i, :] = get_flow1(od, s, dir, num_s)
    return flow


def RMSE(f0, f1, axis=None):
    return np.sqrt(np.mean((f0 - f1) ** 2, axis))


def SMAPE(real, predict):
    a = real.ravel().copy()
    b = predict.ravel().copy()
    mask = ((a>0) & (b>0))
    a = a[mask]
    b = b[mask]
    return 2*np.mean(np.abs(a-b)/(np.abs(a)+np.abs(b)))


def WMAPE(real, predict):
    e = np.sum(np.abs(real - predict))/np.sum(np.abs(real))
    return e


def MAE(real, predict):
    return np.mean(np.abs(real - predict))

def MSE(f0, f1, axis=None):
    return np.mean((f0 - f1) ** 2, axis)


def get_score(real, predict, real_flow, predict_flow):
    print('RMSE of OD: {}'.format(RMSE(real, predict)))
    print('WMAPE of OD: {}'.format(WMAPE(real, predict)))
    print('SMAPE of OD: {}'.format(SMAPE(real, predict)))
    print('MAE of OD: {}'.format(MAE(real, predict)))
    print('r2 of OD: {}'.format(r2_score(real.ravel(), predict.ravel())))
    print('\n')
    print('RMSE of flow: {}'.format(RMSE(real_flow, predict_flow)))
    print('WMAPE of flow: {}'.format(WMAPE(real_flow, predict_flow)))
    print('SMAPE of flow: {}'.format(SMAPE(real_flow, predict_flow)))
    print('MAE of flow: {}'.format(MAE(real_flow, predict_flow)))
    print('r2 of flow: {}'.format(r2_score(real_flow.ravel(), predict_flow.ravel())))


def start_end_idx(start, end, weekend=False, night=False):
    date = pd.period_range('2017-07-01', '2017-09-30 23:30', freq='30T')
    date = date.to_timestamp()
    if not night:
        date = date[date.hour >= 6]
    if not weekend:
        date = date[date.weekday < 5]
    idx = pd.DataFrame(data=np.arange(date.shape[0]), index=date)
    return idx.loc[start:end, :].values.ravel()

# Seperate training and test data

In [10]:
data0 = loadmat('drive//MyDrive//data//OD_3m.mat')
data0 = data0['OD']
data0 = remove_weekends(data0, start=5)
train_idx = start_end_idx('2017-07-03', '2017-08-11', weekend=False, night=False)
test_idx = start_end_idx('2017-08-14', '2017-08-25', weekend=False, night=False)
num_s = 159

# Calculate the HA
data = data0.astype(np.float64)
data_mean = data[:, train_idx].reshape([num_s*num_s, 36, -1], order='F')
data_mean = data_mean.mean(axis=2)

# Subtract the mean of the training set
for i in range(65):
    data[:, i*36:(i+1)*36] = data[:, i*36:(i+1)*36] - data_mean

h = 10
train_data = data[:, train_idx].reshape([159, 159, -1], order='F').transpose([2,0,1])
test_data = data[:, test_idx[0]-h:test_idx[-1]+1].reshape([159, 159, -1], order='F').transpose([2,0,1])

The RMSE of HA is 3.2969849860422897


# Define and train ConvLSTM

In [11]:
class Data(keras.utils.Sequence):
    def __init__(self, data, h=10, batch_size=32, locs=np.array([1,2])):
        # locs is the idx of the x_start location in the data
        self.batch_size = batch_size
        self.h = h
        self.data = data  # (time, O, D)
        self.loc_length = len(locs)
        self.locs = locs
        self.length = int(np.ceil(len(locs) / batch_size))

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        # Only allows positive idx
        if idx < 0:
            raise ValueError('idx must be positive')
        x_start = self.batch_size * idx
        x_end = np.min([self.loc_length, self.batch_size * (idx + 1)])

        batch_x = np.zeros([x_end-x_start, self.h, num_s, num_s, 1], dtype=self.data.dtype)
        batch_y = np.zeros([x_end-x_start, num_s, num_s, 1], dtype=self.data.dtype)
        for i, s in enumerate(range(x_start, x_end)):
            ss = self.locs[s]
            batch_x[i, :, :, :, :] = self.data[ss:(ss+self.h), :, :, np.newaxis]
            batch_y[i, :, :, :] = self.data[ss+self.h, :, :, np.newaxis]

        return (batch_x, batch_y)

trainable_length = train_data.shape[0]-h
reset_random_seeds(1)
random_idx = np.random.permutation(trainable_length)
train_idx = random_idx[0:int(np.floor(trainable_length*0.8))]
validate_idx = random_idx[int(np.floor(trainable_length*0.8)):]

train_Data = Data(train_data, h=10, batch_size=32, locs=train_idx)
validate_Data = Data(train_data, h=10, batch_size=32, locs=validate_idx)

In [12]:
checkpoint_path = 'drive//MyDrive//data//Guangzhou_train_sub_mean10__8(3)_8(3)_1(3)_20210911.ckpt'

call_back = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=20, verbose=0, mode='auto',
    baseline=None, restore_best_weights=True
)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1,
                                                 )

reset_random_seeds(1)
seq = keras.Sequential(
    [
        keras.Input(
            shape=(None, num_s, num_s, 1)
        ),  # Variable-length sequence of num_s x num_s x 1 frames
        layers.ConvLSTM2D(
            filters=8, kernel_size=(3, 3), padding="same", return_sequences=True,
            data_format='channels_last'
        ),
        layers.BatchNormalization(),
        layers.ConvLSTM2D(
            filters=8, kernel_size=(3, 3), padding="same", return_sequences=True,
            data_format='channels_last'
        ),
        layers.BatchNormalization(),
        layers.ConvLSTM2D(
            filters=1, kernel_size=(3, 3), padding="same", return_sequences=False,
            data_format='channels_last'
        ),
    ]
)
seq.compile(loss="mean_squared_error", optimizer="RMSprop")
seq.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d (ConvLSTM2D)    (None, None, 159, 159, 8) 2624      
_________________________________________________________________
batch_normalization (BatchNo (None, None, 159, 159, 8) 32        
_________________________________________________________________
conv_lst_m2d_1 (ConvLSTM2D)  (None, None, 159, 159, 8) 4640      
_________________________________________________________________
batch_normalization_1 (Batch (None, None, 159, 159, 8) 32        
_________________________________________________________________
conv_lst_m2d_2 (ConvLSTM2D)  (None, 159, 159, 1)       328       
Total params: 7,656
Trainable params: 7,624
Non-trainable params: 32
_________________________________________________________________


In [13]:
seq.fit(
    x=train_Data,
    epochs=200,
    steps_per_epoch=len(train_Data),
    verbose=2,
    shuffle=True,
    validation_data=validate_Data,
    validation_steps=len(validate_Data),
    callbacks=[call_back, cp_callback],
)
seq.save_weights(checkpoint_path)

Epoch 1/200
27/27 - 130s - loss: 8.9392 - val_loss: 9.0662

Epoch 00001: saving model to drive//MyDrive//data/Guangzhou_train_sub_mean10__8(3)_8(3)_1(3)_20210911.ckpt
Epoch 2/200
27/27 - 120s - loss: 8.8325 - val_loss: 9.0520

Epoch 00002: saving model to drive//MyDrive//data/Guangzhou_train_sub_mean10__8(3)_8(3)_1(3)_20210911.ckpt
Epoch 3/200
27/27 - 119s - loss: 8.8122 - val_loss: 9.0403

Epoch 00003: saving model to drive//MyDrive//data/Guangzhou_train_sub_mean10__8(3)_8(3)_1(3)_20210911.ckpt
Epoch 4/200
27/27 - 120s - loss: 8.8045 - val_loss: 9.0335

Epoch 00004: saving model to drive//MyDrive//data/Guangzhou_train_sub_mean10__8(3)_8(3)_1(3)_20210911.ckpt
Epoch 5/200
27/27 - 120s - loss: 8.7986 - val_loss: 9.0188

Epoch 00005: saving model to drive//MyDrive//data/Guangzhou_train_sub_mean10__8(3)_8(3)_1(3)_20210911.ckpt
Epoch 6/200
27/27 - 120s - loss: 8.7972 - val_loss: 9.0079

Epoch 00006: saving model to drive//MyDrive//data/Guangzhou_train_sub_mean10__8(3)_8(3)_1(3)_20210911.ckp

In [ ]:
seq.load_weights('/content/drive/MyDrive/data/train_sub_mean10__8(3)_8(3)_1(3)_20210911.ckpt')

# Multi-step forecast

In [15]:
real_OD = data0[:, test_idx]
real_flow = od2flow(real_OD, num_s=num_s)

t_data = data[:, test_idx[0]-h-2:test_idx[-1]+1].reshape([num_s, num_s, -1], order='F').transpose([2,0,1])
n = t_data.shape[0]
results = {step+1: np.zeros((n-h, num_s, num_s)) for step in range(3)}

for i in range(n-h):
    X1 = t_data[np.newaxis, i:i+h, :, :, np.newaxis]
    results[1][i, :, :] = seq.predict(X1).squeeze()
    X2 = np.concatenate([X1[:, 1:, :, :, :], results[1][np.newaxis, [i], :, :, np.newaxis]], axis=1)
    results[2][i, :, :] = seq.predict(X2).squeeze()
    X3 = np.concatenate([X2[:, 1:, :, :, :], results[2][np.newaxis, [i], :, :, np.newaxis]], axis=1)
    results[3][i, :, :] = seq.predict(X3).squeeze()

# 1-step forecast results
predict_OD1 = results[1][2:, :,:].transpose([1,2,0]).reshape([num_s*num_s, -1], order='F')
for i in range(predict_OD1.shape[1]):
    predict_OD1[:, i] += data_mean[:, i%36]
predict_flow1 = od2flow(predict_OD1, num_s=num_s)
print("\n The result of 1-step prediction: \n")
get_score(real_OD, predict_OD1, real_flow, predict_flow1)

# 2-step forecast results
predict_OD2 = results[2][1:-1, :,:].transpose([1,2,0]).reshape([num_s*num_s, -1], order='F')
for i in range(predict_OD2.shape[1]):
    predict_OD2[:, i] += data_mean[:, i%36]
predict_flow2 = od2flow(predict_OD2, num_s=num_s)
print("\n The result of 2-step prediction: \n")
get_score(real_OD, predict_OD2, real_flow, predict_flow2)

# 3-step forecast results
predict_OD3 = results[3][0:-2, :,:].transpose([1,2,0]).reshape([num_s*num_s, -1], order='F')
for i in range(predict_OD3.shape[1]):
    predict_OD3[:, i] += data_mean[:, i%36]
predict_flow3 = od2flow(predict_OD3, num_s=num_s)
print("\n The result of 3-step prediction: \n")
get_score(real_OD, predict_OD3, real_flow, predict_flow3)


 The result of 1-step prediction: 

RMSE of OD: 3.250797308679717
WMAPE of OD: 0.30109549854474443
SMAPE of OD: 0.47551446031588673
MAE of OD: 1.529967464021858
r2 of OD: 0.9507378321115217


RMSE of flow: 117.15764617919922
WMAPE of flow: 0.06868495792150497
SMAPE of flow: 0.10686955600976944
MAE of flow: 55.4928092956543
r2 of flow: 0.9854990497819375

 The result of 2-step prediction: 

RMSE of OD: 3.260425228782025
WMAPE of OD: 0.301791396283843
SMAPE of OD: 0.4759054579824889
MAE of OD: 1.5335035544126245
r2 of OD: 0.9504455994501271


RMSE of flow: 121.21804809570312
WMAPE of flow: 0.07190833240747452
SMAPE of flow: 0.11042226850986481
MAE of flow: 58.097076416015625
r2 of flow: 0.9844764958537556

 The result of 3-step prediction: 

RMSE of OD: 3.2678458145594935
WMAPE of OD: 0.3023349426488549
SMAPE of OD: 0.476302498841124
MAE of OD: 1.5362654962472748
r2 of OD: 0.9502197754458953


RMSE of flow: 123.40123748779297
WMAPE of flow: 0.07345861941576004
SMAPE of flow: 0.111978694

In [16]:
np.savez_compressed('/content/drive/MyDrive/data/Guangzhou_OD_ConvLSTM_step1.npz', data=predict_OD1)
np.savez_compressed('/content/drive/MyDrive/data/Guangzhou_OD_ConvLSTM_step2.npz', data=predict_OD2)
np.savez_compressed('/content/drive/MyDrive/data/Guangzhou_OD_ConvLSTM_step3.npz', data=predict_OD3)